# imports

In [1]:
# 1. 標準庫
import sys
import os
import io
import json
import glob
from typing import Dict
from pathlib import Path

# 2. 第三方套件
import numpy as np
import xarray as xr
import matplotlib.pylab as plt

import tqdm
from IPython.display import clear_output, display, HTML

# 3. 影片相關

# import cv2

# 4. 設定 project 路徑（依你本機路徑調整）
project_root = "C:/Users/GAI/Desktop/Scott/NCA_Research"
if project_root not in sys.path:
    sys.path.append(project_root)

# 5. IPython 魔法指令（Jupyter專用）
%reload_ext autoreload
%autoreload 2

# 6. 共享 core_utils 函式庫導入
from core_utils.plotting import (
    plt_HWC_split_channels,
    plt_CFD_channels,
    plt_random_cfd_slice
)

from core_utils.utils_io import (
    np2pil,      # numpy → PIL Image
    imwrite,     # 儲存圖像為檔案
    imencode,    # 編碼圖像為 byte stream
    im2url,      # 圖像轉 base64 URL（HTML 顯示用）
    load_emoji,   # 載入 emoji 圖像
    load_cfd_npy
)

# from core_utils.utils_image import (
#     imshow,      # 在 notebook 顯示圖像
#     tile2d,      # 多圖拼接
#     zoom         # 放大圖像
# )

# from core_utils.utils_video import (
#     save_video,  # 批次輸出影片
#     VideoWriter  # 逐幀寫入影片（支援 context manager）
# )

# from core_utils.ops_tf_np import (
#     to_rgb,
#     to_rgba,
#     to_alpha,
#     crop_and_resize,
#     get_random_cfd_slices,
#     get_random_cfd_slices_pair
# )


# from core_utils.viz_train import (
#     viz_pool,
#     viz_batch,
#     viz_loss
# )


# 6. 實驗項目 utils 函式庫導入
from E1_basicGNCA.utils.SamplePool import SamplePool

clear_output()

## process urbantales cases into npz file 

In [4]:

# 資料夾路徑
folder = Path("../dataset")

# 存放每個 case 最終 numpy array
case_dict = {}

# 遍歷所有 ped.nc 檔案
for ped_file in folder.rglob("*ped.nc"):
    case_name = ped_file.parent.name
    
    # 讀取 NetCDF
    with xr.open_dataset(ped_file) as ds:
        # 將所有變數堆疊成 3D array (H, W, C)
        arrays = [ds[var].values for var in ds.data_vars]
        ped_np = np.stack(arrays, axis=-1)
    
    # 對應 topo 檔案
    topo_file = ped_file.parent.glob("*_topo")
    topo_file = next(topo_file, None)
    if topo_file:
        topo = np.loadtxt(topo_file)[:, :, np.newaxis]  # (H, W, 1)
        ped_np = np.concatenate([ped_np, topo], axis=2)  # 合併 (H, W, C+1)
    
    case_dict[case_name] = ped_np

# =========================
# 將整個 dict 存成 .npz 檔
# =========================
np.savez_compressed("../dataset/all_cases.npz", **case_dict)

print("所有 case 已存成 all_cases.npz")
print("範例 case shape:", case_dict[next(iter(case_dict))].shape)

ValueError: found the following matches with the input file in xarray's IO backends: ['netcdf4', 'h5netcdf']. But their dependencies may not be installed, see:
https://docs.xarray.dev/en/stable/user-guide/io.html 
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html

In [ ]:
# path = "C:/Users/GAI/Desktop/Scott/NCA_Research/E4_PI-NCA/dataset/FR-PA-V1_d00/FR-PA-V1_d00_topo"

# # ===============================
# # 讀取 topo 檔案
# # ===============================
# topo = np.loadtxt(path)   # shape (576, 576)
# print("Topography shape:", topo.shape)

# # ===============================
# # 2D 視覺化
# # ===============================
# plt.figure(figsize=(7,6))
# plt.imshow(topo, origin='lower', cmap='terrain')
# plt.colorbar(label="Height [m]")
# plt.title("Urban Topography (2D View)")
# plt.xlabel("X grid")
# plt.ylabel("Y grid")
# plt.show()

# # ===============================
# # 3D 視覺化 (保持比例)
# # ===============================
# ny, nx = topo.shape
# X, Y = np.meshgrid(np.arange(nx), np.arange(ny))
# fig = plt.figure(figsize=(10,8))
# ax = fig.add_subplot(111, projection="3d")

# surf = ax.plot_surface(X, Y, topo, cmap="terrain", rstride=2, cstride=2,
#                        linewidth=0, antialiased=True, alpha=1.0)

# ax.set_xlabel("X grid")
# ax.set_ylabel("Y grid")
# ax.set_zlabel("Height [m]")
# ax.set_title("Urban Topography (3D Surface, Equal Aspect)")

# # 🔑 設定 xyz 軸等比例
# max_range = np.array([X.max()-X.min(), Y.max()-Y.min(), topo.max()-topo.min()]).max()
# x_mid = (X.max()+X.min()) * 0.5
# y_mid = (Y.max()+Y.min()) * 0.5
# z_mid = (topo.max()+topo.min()) * 0.5

# ax.set_box_aspect([X.max()-X.min(), Y.max()-Y.min(), topo.max()-topo.min()])  # 等比例

# fig.colorbar(surf, shrink=0.5, aspect=10, label="Height [m]")
# plt.show()

In [ ]:
def create_pool(
    case_dict: Dict[str, np.ndarray],
    poolsize: int,
    hw_size: int = 126
) -> np.ndarray:
    """
    從 case_dict 隨機生成一個 pool，每個樣本為隨機裁剪的 (hw_size, hw_size, 6) 子區域。

    Parameters
    ----------
    case_dict : Dict[str, np.ndarray]
        key: case 名稱，value: numpy array，形狀 (H, W, C)
    poolsize : int
        pool 中樣本數量
    hw_size : int, optional
        每個樣本裁剪的高度與寬度, default=126

    Returns
    -------
    np.ndarray
        shape = (poolsize, hw_size, hw_size, 6)，每個樣本取前 6 個 channel
    """
    pool = []
    case_names = list(case_dict.keys())

    for _ in range(poolsize):
        # 隨機選擇一個 case
        case_name = np.random.choice(case_names)
        arr = case_dict[case_name]
        
        # 確保至少有 6 個 channel
        if arr.shape[2] < 6:
            raise ValueError(f"{case_name} channel < 6")
        
        H, W, _ = arr.shape
        
        # 隨機裁剪起始點
        h_start = np.random.randint(0, H - hw_size + 1)
        w_start = np.random.randint(0, W - hw_size + 1)
        
        # 裁剪子區域，只取前 6 個 channel
        sub = arr[h_start:h_start+hw_size, w_start:w_start+hw_size, :6]
        pool.append(sub)
    
    return np.stack(pool, axis=0)

In [ ]:
# =========================
# 讀取 .npz 檔
# =========================
dict_data = np.load("../dataset/all_cases.npz", allow_pickle=True)

pool = create_pool(dict_data, poolsize=64, hw_size=64)



